In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [2]:
hourly = pd.read_csv('BTC-Hourly.csv')

In [4]:
hourly['date'] = pd.to_datetime(hourly['date'])

In [5]:
hourly = hourly.set_index('date')

In [6]:
hourly.head()

,unix,symbol,open,high,low,close,Volume BTC,Volume USD
date,,,,,,,,
2022-03-01 00:00:00,1646092800,BTC/USD,43221.71,43626.49,43185.48,43312.27,52.056320,2.254677e+06
2022-02-28 23:00:00,1646089200,BTC/USD,43085.30,43364.81,42892.37,43178.98,106.816103,4.612210e+06
2022-02-28 22:00:00,1646085600,BTC/USD,41657.23,44256.08,41650.29,42907.32,527.540571,2.263535e+07
2022-02-28 21:00:00,1646082000,BTC/USD,41917.09,41917.09,41542.60,41659.53,69.751680,2.905822e+06
2022-02-28 20:00:00,1646078400,BTC/USD,41361.99,41971.00,41284.11,41914.97,247.151654,1.035935e+07


In [7]:
hourly['changeBTC'] = hourly['close']-hourly['open']

In [33]:
hourly = hourly.filter(items = ['open','high','low','close','Volume BTC','changeBTC'])

In [34]:
def expSmoothing(data, forecast_periods, alpha):
    data = np.array(data)
    cols = len(data)
    data = np.append(data, [np.nan]*forecast_periods)
    forecast = np.full(cols+forecast_periods, np.nan)
    forecast[1] = data[0]
    for t in range(2,cols+forecast_periods):  
        forecast[t] = alpha*data[t-1]+(1-alpha)*forecast[t-1]
    forecast[cols+1:] = forecast[t]
    df = pd.DataFrame.from_dict({"ActualPrice":data,"ExpSmForecast":forecast,"ExpSmError":data-forecast})
    return df

In [36]:
hourly = hourly.sort_index(ascending = True)
expSmDf = expSmoothing(hourly['close'], 1, 0.3)
hourly = hourly.reset_index()
df = pd.merge(hourly, expSmDf, left_index = True, right_index = True)
df = df.set_index('index')
df = df.sort_index(ascending = False)

In [37]:
df['ExpSmError'].abs().mean()

151.76420614059168